#### Using Hugging Face Transformer

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pandas as pd
from prettytable import PrettyTable 

In [2]:
# topic_queries_relevant_pmcids= {'Nasal Administration': {'queries':
#                  ['COVID-19 Clinical Trials - Efficacy of nasal administration of mucosal Vaccine',
#                 'Efficacy of nasal administration of mucosal Vaccine',
#                 'COVID-19 clinical trials mucosal vaccination aerosol drug delivery maternal immunization nasal administration'],
                
#                     'relevant':
#                     ['PMC2878469', 'PMC7223555', 'PMC5054555', 'PMC4491048']},
                  
#                 'Virus-like particles (VLPs)': {'queries':
#                 ['Efficacy of VLPs-based Vaccines on cancer, allergic and infectious diseases',
#                 'virus-like particles VLPs vaccine production antigen immobilization immune response immunoprophylaxis nanovaccines mucosal VLPs-based vaccines dendritic cells DCs pattern recognition receptors PRRs glycosylations adaptive immune response protection infectious diseases'],
                   
#                     'relevant':
#                     ['PMC7088979', 'PMC7120988', 'PMC7297083', 'PMC7169818']},
                
#                 'Antibiotic': {'queries':
#                 ['Changes in mortality and morbidity caused by different antibiotics on multidrug-resistant organisms',
#                 'bacterial pneumonia diagnostic approaches next-generation sequencing microbial DNA culture-independent definitions of pneumonia antibiotic resistance broad-spectrum antibiotics individualized antibiotic therapy pulmonary infections inflammatory response pneumococcal infection rifampicin beta-lactam antibiotics LTA release inflammatory biomarkers humanized mouse model B cell immortalization human monoclonal antibodies antibiotic pipeline multidrug-resistant organisms antibiotic stewardship critically ill patients nosocomial infections antibiotic therapy pharmacokinetics pharmacodynamics therapeutic drug monitoring fluoroquinolones ICU length of stay'],
                    
#                     'relevant':
#                     ['PMC5850077', 'PMC6640306', 'PMC2949385', 'PMC7080151', 'PMC7277096', 'PMC7080151', 'PMC7303941', 'PMC6969462']},
                
#                 'Marine-derived bacteria': {'queries':
#                 ['Are natural products derived from Marine-sponges effective in treating neoplasm, infectious or autoimmune diseases?',
#                 'marine-derived bacteria natural products salinosporamide A Phase III anticancer clinical trials marine sponge tunicate bacteria-derived natural products bacterial biosynthetic machinery sponge tunicate holobionts metagenomic sequencing synthetic biology marine sponges pharmacologically active compounds secondary metabolites symbiotic microorganisms antiviral agents clinical trials marine sponge Tethya crypta nucleoside Ara-A vidarabine herpes simplex virus HIV metagenomics high throughput microbial cultivation marine drug development marine sponges associated bacteria novel secondary metabolites therapeutic usefulness cancer infection autoimmunity Actinomycetales Sphingomonadales cytokine release peripheral blood mononuclear cells antibacterial antifungal antiprotozoal antiviral activities mechanisms of action'],
                
#                     'relevant':
#                     ['PMC7276339', 'PMC2992996', 'PMC7088305', 'PMC7123853']},

#                 'Traditional Chinese Medicine (TCM)': {'queries':
#                 ['Active ingredients of Traditional Chinese Medicine (TCM) to be used in medicine',
#                 'Traditional Chinese Medicine emergency medicine future potential scientific evidence TCM theoretical framework combined medicine clinical efficacy TCM hospitals China Tanshinones Salvia miltiorrhiza pharmacological activities anti-inflammatory anticancer cardio-cerebrovascular protection drug delivery systems tanshinones liposomes nanoparticles microemulsions cyclodextrin inclusions solid dispersions biological activities TCM prescription multi-component multi-target network pharmacology ligand-protein networks virtual screening in silico target identification ligand-protein networks mode of action TCM'],

#                     'relevant':
#                     ['PMC4129829', 'PMC4708214', 'PMC7120483']},

#                 'extracorporeal membrane oxygenation (ECMO)': {'queries':
#                 ['Utilization of Extracorporeal Membrane Oxygenation (ECMO) for Acute Respiratory Failure',
#                 'indications and contraindications of Extracorporeal Membrane Oxygenation (ECMO)',
#                 'Extracorporeal membrane oxygenation (ECMO) acute respiratory failure indications ECMO management strategies Venovenous veno-arterial ECMO cannulation lung management ECMO VILI ECMO withdrawal prone positioning VV-ECMO safety data oxygenation respiratory system compliance prolonged prone positioning VV-ECMO complications mortality ECMO transportation systematic review pediatric adult populations pooled mortality rates patient transfer referral institution'],
                       
#                     'relevant':
#                     ['PMC7119989', 'PMC4633431', 'PMC5296266']},

#                 'Noninvasive Ventillation': {'queries':
#                 ['Effect of NonInvasive Ventilation (NIV) in patients with Acute Respiratory Failure or Acute Respiratory Distress Syndrome (ARDS)',
#                 'Noninvasive ventilation (NIV) in acute respiratory distress syndrome (ARDS) feasibility, safety, and effectiveness; fiberoptic bronchoscopy (FOB) under NIV in ARDS patients for diagnosis and treatment modification; the impact of prolonged prone positioning during venovenous extracorporeal membrane oxygenation (VV-ECMO) on oxygenation and respiratory system compliance; complications and mortality associated with ECMO transportation; a network meta-analysis comparing the effects of various noninvasive oxygenation strategies on mortality and invasive mechanical ventilation in acute hypoxemic respiratory failure; and the role of NIV in mild to moderate ARDS, focusing on feasibility, success rates, predictors of failure, and outcomes in an Indian population.'],
                        
#                     'relevant':
#                     ['PMC4886426', 'PMC7184712', 'PMC4637959', 'PMC4637959']},

#                 'Parainfluenza Virus': {'queries':
#                 ['Prevalence of different Respiratory virus infection (RVI) in hospitalised patients',
#                 'Respiratory virus infection (RVI) in allogeneic hematopoietic stem cell transplant (allo-HSCT) recipients, exploring the predictive value of the immunodeficiency scoring index (ISI) for severity assessment, and the incidence of pneumonia and outcomes; prevalence, risk factors, and molecular diagnostics of respiratory syncytial virus (RSV) and parainfluenza virus (PIV) among pneumonia patients in Sarawak, Malaysia; the frequency and types of respiratory viruses circulating in Boston long-term care facilities (LTCFs) over a 3-year period, their impact on respiratory illnesses, and the association with lower respiratory tract infections; viral investigations in children attending day-care centers (DCCs) in France, focusing on respiratory syncytial virus (RSV), influenza, parainfluenza, coronavirus, rhinovirus, adenovirus, and enterovirus, and the comparison of viral epidemiological data with or without the use of an immunoactive bacterial extract (OM-85 BV); and the classification of lower respiratory tract disease (LRTD) in patients following hematopoietic cell transplantation (HCT) infected with parainfluenza virus (PIV), assessing the impact on 90-day survival and identifying associated risk factors.'],
                         
#                     'relevant':
#                     ['PMC7169866', 'PMC6093684', 'PMC2875942', 'PMC7182644', 'PMC4001290']},

#                 'H7N9 avian Influenza': {'queries':
#                 ['Efficacy of immunostimulatory therapeutics on H7N9 Avian Influenza Virus',
#                 'Clinical efficacy comparison between oseltamivir-peramivir combination therapy and oseltamivir monotherapy in H7N9 avian influenza patients, analyzing virology, clinical outcomes, and mortality rates; a Phase 1 clinical trial evaluating the safety and immunogenicity of H7 DNA-H7N9 monovalent inactivated vaccine prime-boost regimens in healthy adults; the development and evaluation of a human monoclonal antibody (m826) for H7N9 influenza virus, its binding properties, and protective mechanisms; an Internet-based investigation of the H7N9 outbreak in China, describing epidemiological and clinical characteristics based on publicly available data, and comparing findings with a conventional field epidemiologic study; and an assessment of cross-reactive T-cell immunity to seasonal influenza viruses and human-infecting avian influenza viruses, including H5N1, H6N1, H7N9, and H9N2, in a population presumed to have been previously infected by pandemic H1N1 (pH1N1), exploring potential key conserved epitopes for cross-protection and providing insights for broad-spectrum vaccine development.'],
                          
#                     'relevant':
#                     ['PMC4748590', 'PMC5627236', 'PMC6290738', 'PMC4211021', 'PMC6083907']},

#                 'Interferon‐α2 (IFNα2)': {'queries':
#                 ['Is Interferon-α (IFN-α) suffice for the treatment of Hepatitis B (HBV) infection?',
#                 'Antiviral effects of a single dose of vector-delivered interferon-α1 (AAV-IFNα1) in the context of hepatitis B virus (HBV) treatment, including intracellular and extracellular antigen reduction and decreased viral DNA levels, tested in HBV-producing cells and mice; a novel mode of antiviral action of interferon-α (IFN-α) demonstrated through supernatant from IFN-α-treated cells inhibiting HBV and hepatitis C virus (HCV) entry into hepatoma cells by competing for binding to heparan glycosaminoglycans; and an overview of therapeutic approaches for chronic hepatitis B using IFN-α, nucleos(t)ide analogues, and innovative strategies such as therapeutic vaccination, immunotherapies, and gene therapy tested in the woodchuck (Marmota monax) model.'],
                           
#                     'relevant':
#                     ['PMC7166674', 'PMC5358466', 'PMC3003998']},

#                 'ACE-2': {'queries':
#                 ['Role of Angiotensin-converting enzyme-2 (ACE-2) in different pulmonary disorders',
#                 'Roles and therapeutic implications of angiotensin-converting enzyme 2 (ACE2) in cardiovascular diseases, including its trivalent function, negative regulation of the renin-angiotensin system, facilitation of amino acid transport, and function as the SARS-CoV-2 receptor; examination of the evolutionary relationship between SARS-CoV and SARS-CoV-2, differences in the receptor-binding domain, and the impact on pathogenicity; mechanisms of ACE2 loss of function following SARS-CoV-2 binding; ACE2 as a protective pathway against heart failure, myocardial infarction, hypertension, lung disease, and diabetes mellitus; overview of therapeutic strategies such as recombinant ACE2, gene delivery, Ang 1–7 analogs, and Mas receptor agonists; clinical trials of recombinant human ACE2 (rhACE2) and its effects on plasma angiotensin II and angiotensin 1-7 levels; a decade of research on ACE2 as a regulator of the renin-angiotensin system; and the role of ACE2 in pulmonary arterial hypertension (PAH), its potential as a therapeutic target, and the need for further research on its mechanism in PAH.'],
                            
#                     'relevant':
#                     ['PMC7188049', 'PMC2957877', 'PMC5119454', 'PMC7133233']},

#                 'Flavivirus': {'queries' :
#                 ['Updates on new antiviral discovery for the treatment of genome Flavivirus',
#                 'Review on antiviral research of nucleoside analogs against arthropod-borne Flaviviruses, focusing on inhibitors of flaviviral RNA-dependent RNA polymerase, methyltransferase, and helicase/NTPase, and highlighting challenges in nucleoside analog development and possible strategies to overcome limitations; investigation of arbidol (umifenovir) as a potential broad-spectrum antiviral candidate against arthropod-borne flaviviruses, including Zika virus, West Nile virus, and tick-borne encephalitis virus, assessing anti-viral effects, cell-type dependency, and cytotoxicity profiles; analysis of a small-compound inhibitor, K22, for its potent antiviral activity against Flaviviridae family viruses, particularly focusing on Zika virus (ZIKV), studying time-of-addition experiments, combination regimens with ribavirin (RBV) or interferon alpha (IFN-α), and ultrastructural alterations in ZIKV-induced intracellular replication compartments, suggesting K22 as a potential panviral inhibitor.'],
                
#                     'relevant':
#                     ['PMC5890575', 'PMC5923478', 'PMC6201103']},

#                 'Japanese Encephalitis Virus (JEV)': {'queries':
#                 ['Updates on new therapeutic discovery for the treatment of flavivirus, Japanese Encephalitis Virus (JEV)',
#                 'Antiviral strategies and drug repurposing for Japanese encephalitis virus (JEV) infection, focusing on high-content screening (HCS) of a natural extracts library and the identification of FDA-approved Na(+)/K(+)-ATPase inhibitors (ouabain and digoxin) as efficient against JEV, demonstrating their impact on viral replication and potential as anti-JEV drugs; investigation of Morpholinos as uncharged anti-sense oligomers targeting specific regions of the JEV genome for antiviral and neuroprotective effects in a murine model of JE, highlighting survivability, reduced viral load, protein expression, and cytokine levels, as well as neuroprotection and modulation of stress-related signaling molecules; evaluation of the broad-spectrum antiviral compound arbidol against Zika virus, West Nile virus, and tick-borne encephalitis virus, demonstrating micromolar-level anti-viral effects, cell-type dependence, and potential as a therapeutic agent; and drug repurposing for JEV infection through systems biology methods, identifying 286 genes involved in the progress of JEV infection and suggesting bortezomib as a potential treatment, with evaluation in a JEV-infected mouse model showing reduced lethality and brain damage.'],
                
#                     'relevant':
#                     ['PMC7038234', 'PMC2990691', 'PMC5923478', 'PMC6320907']},
                
#                 'Plasma Therapy': {'queries':
#                 ['Efficacy of Convalescent Plasma therapy on microbial infections',
#                  "Safety, tolerability, and feasibility of plasma infusions in Parkinson's disease (PD) patients, evaluating the impact on cognitive functions, inflammatory markers, uric acid, and quantitative kinematics; assessment of convalescent plasma (CP) for treating Ebola virus disease (EVD), focusing on safety, acceptability, and feasibility, with a call for longer-term follow-up and scalability evaluation; exploration of convalescent plasma (CP) as a potential treatment for coronavirus disease 2019 (COVID-19), emphasizing the lack of proven treatments and the need for formal proof of efficacy, summarizing historical clinical trials and recommending cautious use in critically ill patients; investigation of cold atmospheric argon plasma treatment in chronic infected wounds, comparing a 2-minute treatment with different plasma devices, assessing safety, bacterial load reduction, and efficacy in healing chronic wounds."],
                 
#                     'relevant':
#                     ['PMC7361360', 'PMC4678103', 'PMC7314270', 'PMC7161860']},

#                 'Procalcitonin': {'queries' :
#                 ['Is serum Procalcitonin (PCT) an accurate predictor to safely reduce antibiotic therapy in respiratory tract infections?',
#                  'Evaluation of factors influencing adherence to procalcitonin (PCT) treatment algorithms in a randomized clinical trial (RCT) for lower respiratory tract infections (LRTI), focusing on algorithm compliance, provider antibiotic prescribing practices, and surveying provider knowledge and attitudes towards biomarkers and viral testing; Examination of the association between serum procalcitonin concentration at hospital admission and pathogens detected in adults hospitalized with community-acquired pneumonia, assessing the accuracy of procalcitonin to discriminate between viral and bacterial pneumonia; Investigation of serum procalcitonin levels as a biomarker to guide antibiotic therapy in patients with chronic obstructive pulmonary disease (COPD), with a focus on predicting viral versus bacterial infection in acute exacerbations of COPD (AECOPD) with and without pneumonia, and comparing clinical and laboratory features in subjects with viral and bacterial diagnoses.'],
                 
#                     'relevant':
#                     ['PMC4820114', 'PMC5850442', 'PMC3292390']},

#                 'Organ Transplantation': {'queries':
#                 ['Complications in patients undergoing solid organ transplantation',
#                 'Updates on prevention of major healthcare-associated infections unique to transplantation, emergence of antimicrobial resistant infections, and strategies for identifying and preventing transmission of these pathogens in transplant recipients; Review of biomaterials and implants addressing infection and inflammation issues for compromised patients undergoing organ regeneration; Discussion of invasive fungal infections (IFIs) as a significant complication of solid organ transplantation, including risk factors, diagnosis challenges, and antifungal treatment strategies; Overview of the changing landscape of donation after circulatory death (DCD) liver transplantation (LT), including improvements in outcomes, challenges related to ischemic cholangiopathy, and emerging technologies for reducing complications in DCD LT.'],
                 
#                     'relevant':
#                     ['PMC7088673', 'PMC5016531', 'PMC7138456', 'PMC7357263']},

#                 'Acute Otitis Media': {'queries':
#                 ['Studies to identify different aetiologies of Acute Otitis Media (OAM) in children',
#                 'Impact of influenza vaccine on childhood otitis media; Retrospective cohort study with 803,592 children (<10 years old) in Taiwan; Evaluation of acute otitis media (AOM) and tympanostomy tube insertion incidence; Comparison of seasons before and after the introduction of traditional injectable trivalent influenza vaccine (TIV); Significant reduction in seasonal AOM incidence in children < 2 years old after TIV introduction in 2004; Decline in outpatient visits for AOM in influenza seasons of 2005 and 2006 with increased vaccine coverage; Observations on direct protection and potential herd immunity induced by TIV; Reductions observed only in vaccine-eligible children, while older children experienced increases in AOM incidence; Association between respiratory viral infection, bacterial density, and progression to AOM with perforation; Retrospective examination of 366 nasopharyngeal swabs from 114 Indigenous children; Screening for 17 respiratory viruses and estimating bacterial densities (S. pneumoniae, H. influenzae, M. catarrhalis); Positive association between nasopharyngeal bacterial load, clinical ear state, and exacerbation by respiratory viruses; Independent association of human adenovirus (HAdV) with acute ear states; Elucidation of clinical outcome and etiology of acute otitis media (AOM) in children; Study group of 120 children aged 6 to 144 months with AOM; Testing middle ear fluid (MEF) for viral pathogens (RSV, HRV, HCV, IV type A, AV, PIV type -3) and bacteria (S. pneumoniae, H. influenzae); Assessment of clinical response on days 2 to 4, 11 to 13, and 26 to 28; Identification of RSV as the most common virus in MEF samples, followed by HRV, HCV, IV type A, AV, and PIV type -3; Isolation of 69 bacterial species, with S. pneumoniae being the most frequently identified; Detection of viral RNA in both bacteria-negative and bacteria-positive MEF samples; Analysis of clinical cure, relapse, reinfection rates among children with viral infection alone, combined viral and bacterial infection, bacterial infec'],
                  
#                     'relevant':
#                     ['PMC5755876', 'PMC3128050', 'PMC7086696']},

#                 'Cardiovascular': {'queries':
#                 ['Correlation of lipids and Angiotensin-converting enzyme 2 (ACE2) on Cardiovascular Health',
#                 'Effect of breakfast fat composition on lipid metabolism and cardiovascular risk (CVR) in women; Comparison of polyunsaturated fatty acid (PUFA)rich, saturated fatty acid (SFA)-rich, and monounsaturated fatty acid (MUFA)-rich breakfasts in a randomized crossover clinical trial; Assessment of plasma lipoprotein and apolipoprotein subfractions; Role of angiotensin-converting enzyme 2 (ACE2) in the renin–angiotensin system and its connection to cardiovascular (CV) disease, coronaviruses, and influenza viruses; Postulation on the relationship between high ACE2 expression in high CV risk populations and susceptibility to coronavirus and influenza virus infections; Potential strategy of using coronavirus and influenza virus vaccines in high CV risk populations to prevent CV disease and viral infections; Review of current pharmacologic strategies based on renin angiotensin system (RAS) blockade for CV disease treatment; Exploration of the ACE2/Ang(1–9) axis as a target to counterbalance the vasoconstrictive/proliferative RAS axis and protect against adverse cardiovascular remodeling in hypertension and heart failure; Summary of selected studies on cardiovascular disease (CVD) prevention presented at the American College of Cardiology 2020 Virtual Scientific Session (ACC.20)/World Cardiology Congress (WCC); Inclusion of clinical trials on alirocumab, evinacumab, renal denervation, vericiguat, and inclisiran; Analysis of eicosapentaenoic acid levels and cardiovascular outcomes from the Reduction of Cardiovascular Events with Icosapent Ethyl-Intervention Trial (REDUCE-IT); Examination of the use of polygenic risk score with low density lipoprotein cholesterol (LDL-C) and systolic blood pressure (SBP) on lifetime cardiovascular risk.'],
                
#                     'relevant':
#                     ['PMC7352537', 'PMC7184394', 'PMC3270559', 'PMC7301047']},

#                 'Bronchopulmonary dysplasia': {'queries':
#                 ['Prevention strategies to avoid Bronchopulmonary Dysplasia (BPD) in neonates',
#                 '"Bronchopulmonary Dysplasia (BPD)" "prematurity complication" "BPD prevention" "oxygen management" "noninvasive ventilation strategies" "best practice guidelines" "Shewhart control charts" "0.21 by 28 campaign" "reduction in the any BPD rate" "decrease in severe BPD" "multidisciplinary QI initiative" "process improvement" "pathophysiological contributors of BPD" "reduced rate of BPD" "referral level IV NICU"'],
                
#                     'relevant':
#                     ['PMC6920537', 'PMC7122497', 'PMC7223819']},

#                 'Leukemia': {'queries':
#                 ['Recent developments in leukemia and related therapies, including targeted treatments, infections, CAR T cell therapy, intensive cytotoxic therapy in infants, and reduced-intensity therapy with decitabine'],
                
#                     'relevant':
#                     ['PMC7178857', 'PMC5907417', 'PMC7096894', 'PMC7101778', 'PMC5727246']},

#                 'Bronchiolitis': {'queries' :
#                 ['Treatment approaches for bronchiolitis in infants',
#                 'Pediatric bronchiolitis treatment approaches, including high-flow nasal cannula (HFNC) feasibility in a general ward, chronic lung allograft dysfunction (CLAD) in lung transplant recipients, factors influencing hospitalization outcomes in infants with bronchiolitis, impact of bronchiolitis on extrapulmonary systems and available therapeutic options, and the effectiveness and safety of complementary and alternative medicine (CAM) in infants under 2 years with bronchiolitis.'],
                
#                     'relevant':
#                     ['PMC7087157', 'PMC7122385', 'PMC7167668', 'PMC7120754', 'PMC5315308']},

#                 'Masks and Respirators': {'queries':
#                 ['Efficacy of medical masks and respirators in protecting against respiratory infections in Healthcare Workers',
#                 'Comparing mask and respirator use in healthcare workers: guidelines variations, effectiveness, and cost-effectiveness. This includes evaluating the WHO and CDC guidelines, examining efficacy, acceptability, and tolerability of non-pharmaceutical interventions in households with influenza index patients, assessing the cost-effectiveness of N95 respirators and medical masks for healthcare workers in Beijing, China, and examining the efficacy of medical masks and respirators in protecting against respiratory infections using pooled data from two randomized control clinical trials.'],
                
#                     'relevant':
#                     ['PMC3693993', 'PMC3285078', 'PMC5496227', 'PMC5705692']},
                    
#                     }


In [2]:
topic_queries_relevant_pmcids= {'Nasal Administration': {'query':
                 'COVID-19 Clinical Trials - Efficacy of nasal administration of mucosal Vaccine',
                
                    'relevant':
                    ['PMC2878469', 'PMC7223555', 'PMC5054555', 'PMC4491048']},
                  
                'Virus-like particles (VLPs)': {'query':
                'Efficacy of VLPs-based Vaccines on cancer, allergic and infectious diseases',
                   
                    'relevant':
                    ['PMC7088979', 'PMC7120988', 'PMC7297083', 'PMC7169818']},
                
                # 'Antibiotic': {'query':
                # ['Changes in mortality and morbidity caused by different antibiotics on multidrug-resistant organisms',
                # 'bacterial pneumonia diagnostic approaches next-generation sequencing microbial DNA culture-independent definitions of pneumonia antibiotic resistance broad-spectrum antibiotics individualized antibiotic therapy pulmonary infections inflammatory response pneumococcal infection rifampicin beta-lactam antibiotics LTA release inflammatory biomarkers humanized mouse model B cell immortalization human monoclonal antibodies antibiotic pipeline multidrug-resistant organisms antibiotic stewardship critically ill patients nosocomial infections antibiotic therapy pharmacokinetics pharmacodynamics therapeutic drug monitoring fluoroquinolones ICU length of stay'],
                    
                #     'relevant':
                #     ['PMC5850077', 'PMC6640306', 'PMC2949385', 'PMC7080151', 'PMC7277096', 'PMC7080151', 'PMC7303941', 'PMC6969462']},
                
                'Marine-derived bacteria': {'query':
                'Are natural products derived from Marine-sponges effective in treating neoplasm, infectious or autoimmune diseases?',
                
                    'relevant':
                    ['PMC7276339', 'PMC2992996', 'PMC7088305', 'PMC7123853']},

                'Traditional Chinese Medicine (TCM)': {'query':
                'Active ingredients of Traditional Chinese Medicine (TCM) to be used in medicine',
                
                    'relevant':
                    ['PMC4129829', 'PMC4708214', 'PMC7120483']},

                # 'extracorporeal membrane oxygenation (ECMO)': {'query':
                # ['Utilization of Extracorporeal Membrane Oxygenation (ECMO) for Acute Respiratory Failure',
                # 'indications and contraindications of Extracorporeal Membrane Oxygenation (ECMO)',
                # 'Extracorporeal membrane oxygenation (ECMO) acute respiratory failure indications ECMO management strategies Venovenous veno-arterial ECMO cannulation lung management ECMO VILI ECMO withdrawal prone positioning VV-ECMO safety data oxygenation respiratory system compliance prolonged prone positioning VV-ECMO complications mortality ECMO transportation systematic review pediatric adult populations pooled mortality rates patient transfer referral institution'],
                       
                #     'relevant':
                #     ['PMC7119989', 'PMC4633431', 'PMC5296266']},

                'Noninvasive Ventillation': {'query':
                'Effect of NonInvasive Ventilation (NIV) in patients with Acute Respiratory Failure or Acute Respiratory Distress Syndrome (ARDS)',
                       
                    'relevant':
                    ['PMC4886426', 'PMC7184712', 'PMC4637959', 'PMC4637959']},

                'Parainfluenza Virus': {'query':
                'Prevalence of different Respiratory virus infection (RVI) in hospitalised patients',
                                         
                    'relevant':
                    ['PMC7169866', 'PMC6093684', 'PMC2875942', 'PMC7182644', 'PMC4001290']},

                'H7N9 avian Influenza': {'query':
                'Efficacy of immunostimulatory therapeutics on H7N9 Avian Influenza Virus',
                          
                    'relevant':
                    ['PMC4748590', 'PMC5627236', 'PMC6290738', 'PMC4211021', 'PMC6083907']},

                'Interferon‐α2 (IFNα2)': {'query':
                'Is Interferon-α (IFN-α) suffice for the treatment of Hepatitis B (HBV) infection?',
                                           
                    'relevant':
                    ['PMC7166674', 'PMC5358466', 'PMC3003998']},

                'ACE-2': {'query':
                'Role of Angiotensin-converting enzyme-2 (ACE-2) in different pulmonary disorders',
                                            
                    'relevant':
                    ['PMC7188049', 'PMC2957877', 'PMC5119454', 'PMC7133233']},

                'Flavivirus': {'query' :
                'Updates on new antiviral discovery for the treatment of genome Flavivirus',
                                
                    'relevant':
                    ['PMC5890575', 'PMC5923478', 'PMC6201103']},

                'Japanese Encephalitis Virus (JEV)': {'query':
                'Updates on new therapeutic discovery for the treatment of flavivirus, Japanese Encephalitis Virus (JEV)',
                                
                    'relevant':
                    ['PMC7038234', 'PMC2990691', 'PMC5923478', 'PMC6320907']},
                
                'Plasma Therapy': {'query':
                'Efficacy of Convalescent Plasma therapy on microbial infections',
                                 
                    'relevant':
                    ['PMC7361360', 'PMC4678103', 'PMC7314270', 'PMC7161860']},

                'Procalcitonin': {'query' :
                'Is serum Procalcitonin (PCT) an accurate predictor to safely reduce antibiotic therapy in respiratory tract infections?',
                                 
                    'relevant':
                    ['PMC4820114', 'PMC5850442', 'PMC3292390']},

                'Organ Transplantation': {'query':
                'Complications in patients undergoing solid organ transplantation',
                                 
                    'relevant':
                    ['PMC7088673', 'PMC5016531', 'PMC7138456', 'PMC7357263']},

                'Acute Otitis Media': {'query':
                'Studies to identify different aetiologies of Acute Otitis Media (OAM) in children',
                                  
                    'relevant':
                    ['PMC5755876', 'PMC3128050', 'PMC7086696']},

                'Cardiovascular': {'query':
                'Correlation of lipids and Angiotensin-converting enzyme 2 (ACE2) on Cardiovascular Health',
                                
                    'relevant':
                    ['PMC7352537', 'PMC7184394', 'PMC3270559', 'PMC7301047']},

                'Bronchopulmonary dysplasia': {'query':
                'Prevention strategies to avoid Bronchopulmonary Dysplasia (BPD) in neonates',
                                
                    'relevant':
                    ['PMC6920537', 'PMC7122497', 'PMC7223819']},

                'Leukemia': {'query':
                'Recent developments in leukemia and related therapies, including targeted treatments, infections, CAR T cell therapy, intensive cytotoxic therapy in infants, and reduced-intensity therapy with decitabine',
                
                    'relevant':
                    ['PMC7178857', 'PMC5907417', 'PMC7096894', 'PMC7101778', 'PMC5727246']},

                'Bronchiolitis': {'query' :
                'Treatment approaches for bronchiolitis in infants',
                                
                    'relevant':
                    ['PMC7087157', 'PMC7122385', 'PMC7167668', 'PMC7120754', 'PMC5315308']},

                'Masks and Respirators': {'query':
                'Efficacy of medical masks and respirators in protecting against respiratory infections in Healthcare Workers',
                                
                    'relevant':
                    ['PMC3693993', 'PMC3285078', 'PMC5496227', 'PMC5705692']},
                    
                    }


In [3]:
topic_queries_relevant_pmcids['Nasal Administration']

{'query': 'COVID-19 Clinical Trials - Efficacy of nasal administration of mucosal Vaccine',
 'relevant': ['PMC2878469', 'PMC7223555', 'PMC5054555', 'PMC4491048']}

In [4]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")

In [5]:
relevant_papers_df = pd.read_csv('../relevant_papers_of_query_dataset.csv')
relevant_papers_df

,pmcid,title,abstract
0,PMC2990691,Antiviral and Neuroprotective Role of Octaguan...,"BACKGROUND: Japanese encephalitis (JE), caused..."
1,PMC3003998,Therapeutic Vaccination in Chronic Hepatitis B...,Recommended treatment of chronic hepatitis B w...
2,PMC3270559,Protective Role of the ACE2/Ang-(1–9) Axis in ...,Despite reduction in cardiovascular (CV) event...
3,PMC4001290,Parainfluenza Virus Lower Respiratory Tract Di...,Background. Parainfluenza virus (PIV) commonly...
4,PMC4129829,The potential contributions of traditional Chi...,BACKGROUND: Despite the fact that traditional ...
...,...,...,...
65,PMC7086696,Acute otitis media and respiratory viruses,The present study was performed to elucidate t...
66,PMC6201103,The Small-Compound Inhibitor K22 Displays Broa...,The virus family Flaviviridae encompasses seve...
67,PMC7184394,The role of angiotensin-converting enzyme 2 in...,Angiotensin-converting enzyme 2 (ACE2) has eme...
68,PMC4678103,The Use of Ebola Convalescent Plasma to Treat ...,The clinical evaluation of convalescent plasma...


In [6]:
papers_to_remove_to_avoid_truncation = ['PMC4211021','PMC4633431','PMC7182644']

In [7]:
relevant_papers_df = relevant_papers_df[~relevant_papers_df['pmcid'].isin(papers_to_remove_to_avoid_truncation)]
relevant_papers_df

,pmcid,title,abstract
0,PMC2990691,Antiviral and Neuroprotective Role of Octaguan...,"BACKGROUND: Japanese encephalitis (JE), caused..."
1,PMC3003998,Therapeutic Vaccination in Chronic Hepatitis B...,Recommended treatment of chronic hepatitis B w...
2,PMC3270559,Protective Role of the ACE2/Ang-(1–9) Axis in ...,Despite reduction in cardiovascular (CV) event...
3,PMC4001290,Parainfluenza Virus Lower Respiratory Tract Di...,Background. Parainfluenza virus (PIV) commonly...
4,PMC4129829,The potential contributions of traditional Chi...,BACKGROUND: Despite the fact that traditional ...
...,...,...,...
65,PMC7086696,Acute otitis media and respiratory viruses,The present study was performed to elucidate t...
66,PMC6201103,The Small-Compound Inhibitor K22 Displays Broa...,The virus family Flaviviridae encompasses seve...
67,PMC7184394,The role of angiotensin-converting enzyme 2 in...,Angiotensin-converting enzyme 2 (ACE2) has eme...
68,PMC4678103,The Use of Ebola Convalescent Plasma to Treat ...,The clinical evaluation of convalescent plasma...


In [8]:
# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0] # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)

def compute_embeddings(list_of_docs, tokenizer, model):
    # Tokenize sentences
    inputs = tokenizer(list_of_docs, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        output = model(**inputs)

    # Perform pooling. In this case, mean pooling.
    embeddings = meanpooling(output, inputs['attention_mask'])

    return embeddings

In [9]:
relevant_papers_embeddings = compute_embeddings(relevant_papers_df['abstract'].to_list(), tokenizer=tokenizer, model=model)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
relevant_papers_embeddings.shape

torch.Size([68, 768])

In [19]:
pmcids_list = relevant_papers_df['pmcid'].to_list()
for topic in topic_queries_relevant_pmcids:
    print(topic)
    
    # topic_queries_relevant_pmcids[topic]['precision@4'] = []
    query = topic_queries_relevant_pmcids[topic]['query']
        
    count = 0
    query_embedding = compute_embeddings(query, tokenizer, model)
    similarity_scores = F.cosine_similarity(query_embedding,relevant_papers_embeddings)
    sort_similarities = similarity_scores.argsort(descending=True).tolist()[:4]
    
    for index in sort_similarities:
        if pmcids_list[index] in topic_queries_relevant_pmcids[topic]['relevant']:
            count += 1
    topic_queries_relevant_pmcids[topic]['count'] = count
    topic_queries_relevant_pmcids[topic]['precision@4'] = round(count/4,2) if len(topic_queries_relevant_pmcids[topic]['relevant']) >= 4 else round(count/len(topic_queries_relevant_pmcids[topic]['relevant']),2)

Nasal Administration
Virus-like particles (VLPs)
Marine-derived bacteria
Traditional Chinese Medicine (TCM)
Noninvasive Ventillation
Parainfluenza Virus
H7N9 avian Influenza
Interferon‐α2 (IFNα2)
ACE-2
Flavivirus
Japanese Encephalitis Virus (JEV)
Plasma Therapy
Procalcitonin
Organ Transplantation
Acute Otitis Media
Cardiovascular
Bronchopulmonary dysplasia
Leukemia
Bronchiolitis
Masks and Respirators


In [26]:
# Specify the Column Names while initializing the Table 
myTable = PrettyTable(["Topic", "Query", "Matched", "Total", "precision@4"])

for topic in topic_queries_relevant_pmcids:
    myTable.add_row([topic, topic_queries_relevant_pmcids[topic]['query'][:20]+"....", topic_queries_relevant_pmcids[topic]['count'], len(topic_queries_relevant_pmcids[topic]['relevant']), topic_queries_relevant_pmcids[topic]['precision@4']])

print(myTable) 


+------------------------------------+--------------------------+---------+-------+-------------+
|               Topic                |          Query           | Matched | Total | precision@4 |
+------------------------------------+--------------------------+---------+-------+-------------+
|        Nasal Administration        | COVID-19 Clinical Tr.... |    2    |   4   |     0.5     |
|    Virus-like particles (VLPs)     | Efficacy of VLPs-bas.... |    3    |   4   |     0.75    |
|      Marine-derived bacteria       | Are natural products.... |    4    |   4   |     1.0     |
| Traditional Chinese Medicine (TCM) | Active ingredients o.... |    3    |   3   |     1.0     |
|      Noninvasive Ventillation      | Effect of NonInvasiv.... |    3    |   4   |     0.75    |
|        Parainfluenza Virus         | Prevalence of differ.... |    3    |   5   |     0.75    |
|        H7N9 avian Influenza        | Efficacy of immunost.... |    4    |   5   |     1.0     |
|       Interferon‐α

In [25]:
sum_precision = 0
for topic in topic_queries_relevant_pmcids:
    sum_precision += topic_queries_relevant_pmcids[topic]['precision@4']

print("Average Precision ",round(sum_precision/20,2))

Average Precision  0.81


In [27]:
# to set the value of k 

import statistics

total_relevant_papers_in_each_query = []
for topic in topic_queries_relevant_pmcids:
    total_relevant_papers_in_each_query.append(len(topic_queries_relevant_pmcids[topic]['relevant']))

statistics.mode(total_relevant_papers_in_each_query)

4

In [15]:
topic_queries_relevant_pmcids

{'Nasal Administration': {'queries': ['COVID-19 Clinical Trials - Efficacy of nasal administration of mucosal Vaccine',
   'Efficacy of nasal administration of mucosal Vaccine',
   'COVID-19 clinical trials mucosal vaccination aerosol drug delivery maternal immunization nasal administration'],
  'relevant': ['PMC2878469', 'PMC7223555', 'PMC5054555', 'PMC4491048'],
  'count': [2, 3, 4]},
 'Virus-like particles (VLPs)': {'queries': ['Efficacy of VLPs-based Vaccines on cancer, allergic and infectious diseases',
   'virus-like particles VLPs vaccine production antigen immobilization immune response immunoprophylaxis nanovaccines mucosal VLPs-based vaccines dendritic cells DCs pattern recognition receptors PRRs glycosylations adaptive immune response protection infectious diseases'],
  'relevant': ['PMC7088979', 'PMC7120988', 'PMC7297083', 'PMC7169818'],
  'count': [3, 3]},
 'Antibiotic': {'queries': ['Changes in mortality and morbidity caused by different antibiotics on multidrug-resistant

In [93]:
query = 'COVID-19 Clinical Trials - Efficacy of nasal administration of mucosal Vaccine'

query_embedding = compute_embeddings(query, tokenizer, model)

In [94]:
similarity_scores = F.cosine_similarity(query_embedding,relevant_papers_embeddings)

In [95]:
sort_similarities = similarity_scores.argsort(descending=True).tolist()[:10]

In [96]:
sort_similarities

[30, 50, 11, 9, 45, 49, 74, 56, 44, 63]

In [97]:
sort_similarities[0]

30

In [100]:
for index in sort_similarities:
    if relevant_papers_df['pmcid'][index] in topic_queries_relevant_pmcids['Nasal Administration']['relevant']:
        print('True')

In [13]:

relevant_papers_df['pmcid'][67]

'PMC5119454'

In [14]:
relevant_papers_df['pmcid'].to_list()

['PMC2949385',
 'PMC2990691',
 'PMC3003998',
 'PMC3270559',
 'PMC4001290',
 'PMC4129829',
 'PMC4748590',
 'PMC4886426',
 'PMC5296266',
 'PMC5627236',
 'PMC5727246',
 'PMC5755876',
 'PMC5850077',
 'PMC5907417',
 'PMC6083907',
 'PMC6920537',
 'PMC6969462',
 'PMC7087157',
 'PMC7096894',
 'PMC7101778',
 'PMC7120483',
 'PMC7161860',
 'PMC7184712',
 'PMC7223555',
 'PMC7223819',
 'PMC7303941',
 'PMC7123853',
 'PMC7357263',
 'PMC6290738',
 'PMC7167668',
 'PMC2878469',
 'PMC5315308',
 'PMC5358466',
 'PMC5923478',
 'PMC5496227',
 'PMC5850442',
 'PMC7301047',
 'PMC3128050',
 'PMC2957877',
 'PMC7133233',
 'PMC3693993',
 'PMC3292390',
 'PMC7080151',
 'PMC7277096',
 'PMC7088979',
 'PMC5705692',
 'PMC7276339',
 'PMC7088305',
 'PMC7188049',
 'PMC3285078',
 'PMC4491048',
 'PMC4708214',
 'PMC7169866',
 'PMC7352537',
 'PMC6640306',
 'PMC4820114',
 'PMC7169818',
 'PMC2992996',
 'PMC5054555',
 'PMC5016531',
 'PMC7166674',
 'PMC4637959',
 'PMC7088673',
 'PMC7314270',
 'PMC6320907',
 'PMC5119454',
 'PMC28759

In [15]:
relevant_papers_df['abstract'].to_list()

['BACKGROUND: Passive transfer of antibodies not only provides immediate short-term protection against disease, but also can be exploited as a therapeutic tool. However, the ‘humanization’ of murine monoclonal antibodies (mAbs) is a time-consuming and expensive process that has the inherent drawback of potentially altering antigenic specificity and/or affinity. The immortalization of human B cells represents an alternative for obtaining human mAbs, but relies on the availability of biological samples from vaccinated individuals or convalescent patients. In this work we describe a novel approach to generate fully human mAbs by combining a humanized mouse model with a new B cell immortalization technique. METHODOLOGY/PRINCIPAL FINDINGS: After transplantation with CD34(+)CD38(−) human hematopoietic progenitor cells, BALB/c Rag2(−/−)IL-2Rγc(−/−) mice acquire a human immune system and harbor B cells with a diverse IgM repertoire. “Human Immune System” mice were then immunized with two comme